# Predicting NFL Plays Based on Historical Play-by-Play Data

The NFL releases play-by-play data in a single text file with one row for each play. The row can contain up to ~60 discrete pieces of information, though the order and inclusion (or exclusion) of each piece of information will vary. Some basic parameters that are always included for each row include the time, quarter, offense/defense teams, down, yard line, main players involved (quarterback/receivers if passing play, running back if running play). The amount of information provided can vary greatly, as some plays are much more complicated than others, for example:

(10:26) (SHOTGUN) 3-J.WINSTON PASS SHORT LEFT TO 13-M.EVANS TO TB 49 FOR 8 YARDS (34-A.BOUYE). FUMBLES (34-A.BOUYE), RECOVERED BY HOU-56-B.CUSHING AT TB 49. 56-B.CUSHING TO TB 44 FOR 5 YARDS (70-L.MANKINS). HOUSTON CHALLENGED THE PASS COMPLETION RULING, AND THE PLAY WAS REVERSED. (SHOTGUN) 3-J.WINSTON PASS INCOMPLETE SHORT LEFT TO 13-M.EVANS.

Compared to:

(6:17) (SHOTGUN) 15-R.MALLETT PASS SHORT MIDDLE TO 10-D.HOPKINS TO 50 FOR 6 YARDS (50-B.CARTER).

Fortunately, a team of sports data analysts have compiled a database that has cleaned this information. They've processed this data and created tables that separate out all possible parameters of a single play, and record all this information on a single line that can be mapped to by a unique Play ID.

Using this data I'd like to predict the type of play a team will carry out based on certain parameters. Initially, the parameters I believe would be most useful are:

1. Down (1st, 2nd, 3rd, 4th)
2. Yards to go
3. Field Position
4. Team (certain teams will have more of a run or pass focus depending on their type of offense)

In [2]:
#Importing necessary packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split
from sklearn import metrics
import statsmodels.formula.api as smf

In [ ]:
#Reading in data
all_data = pd.read_csv('PBP.csv')

To begin, looked at just 2014 data to identify basic trends

In [ ]:
pbp_2014 = all_data[all_data.gid > 3722]

In [ ]:
#Example team, grouping by play type and down, only Pass and Rush
pbp_2014[pbp_2014.off == 'GB'][(pbp_2014.type == 'RUSH') | (pbp_2014.type == 'PASS')].groupby([pbp_2014.dwn,
 pbp_2014.type]).pid.count().plot(kind = 'bar')

In [ ]:
## Running this will create 32 graphs (one for each team), similar to the one above
for row in pbp_2014.off.unique():
   fig = plt.figure()    
   team = pbp_2014[pbp_2014.off == row][(pbp_2014.type == 'RUSH') | (pbp_2014.type == 'PASS')].groupby([pbp_2014.dwn, pbp_2014.type]).pid.count()
   team.plot(axes=fig, kind='bar', title = row) 

# K-Nearest Neighbor

I only included run and pass plays for this part of the analysis (but could use all play types if there's a good reason). I did this because, at this point, I'm most interested in seeing if a run or pass play will occur, rather than punt, FG, or kickoffs. These data points would skew the results because a team might choose to kick a field goal (or punt) rather than run a play depending on a number of other factors outside the scope of this analysis. As such, the below line creates a new data frame consisting of only run and pass plays.

In [ ]:
pbp_2014_RP = pbp_2014[(pbp_2014.type == 'RUSH') | (pbp_2014.type == 'PASS')]

Using down, yards, and field position as independent variables, I ran KNN for a sample team below.

In [67]:
X,y = pbp_2014_RP[['dwn', 'ytg', 'yfog']], pbp_2014_RP['type']
knn = KNeighborsClassifier(n_neighbors=5)           
knn.fit(X, y)                                       
knn.score(X, y)

0.66355828220858892

I played around testing a few scenarios to see if this model intuitively makes sense. If a team is facing a 2nd & 1 on the opponent's one-yard line, most people would predict the team would run the ball. KNN tells us this as well:

In [68]:
X_new = [2, 1, 99]
knn.predict(X_new)

array(['RUSH'], dtype=object)

Similarly, in most 3rd & 10 situations, a team will pass. Will this change depending on their field position?

In [69]:
Y_new = [3, 10, 1]
knn.predict(Y_new)

array(['RUSH'], dtype=object)

In [70]:
Z_new = [3, 10, 40]
knn.predict(Z_new)

array(['PASS'], dtype=object)

A team is more likely to rush on 3rd & 10 when they are on their own 1 yard line. This makes sense, as passes are more prone to turnovers, which would give the opposing team good field position. In contrast, on 3rd and 10 on a team's own 40 yard line, KNN tells us they would pass. We visualize the ideal K below

#Note to self - need to look at "Score" versus accuracy, getting different values

In [ ]:
from sklearn.grid_search import GridSearchCV
knn = KNeighborsClassifier()
k_range = range(1, 30, 2)
param_grid = dict(n_neighbors=k_range)
grid = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')
grid.fit(X, y)

grid.grid_scores_
grid_mean_scores = [result[1] for result in grid.grid_scores_]

# plotting the results
plt.figure()
plt.plot(k_range, grid_mean_scores)

#Logistic Regression

As we're only considering a standard play where a team will run or pass (and not kick, like at the end of a game or on 4th down), we can run a linear regression and figure out the probability that a team will run or not run. We are able to do this because we are only considering two outcomes (yes = run, no = pass). I created 2 additional columns 'rush' and 'pass' and they contain 0 or 1 depending on if the play is a rush or pass, shown below. This allows us to interpret our logistic regression using integers instead of text.

In [ ]:
pbp_2014_RP['rush'] = pbp_2014_RP.type.map({'RUSH': 1, 'PASS' : 0})
pbp_2014_RP['pass'] = pbp_2014_RP.type.map({'RUSH': 0, 'PASS' : 1})

In [ ]:
##The below logistic regression gives us:

# intercept = 1.83365024
#'dwn', -0.70040617414199391),
#'ytg', -0.10659136716341924),
#'yfog', -0.00086277878476188332)


logreg = LogisticRegression(C=1e9) 
feature_cols = ['dwn', 'ytg', 'yfog']
X = pbp_2014_RP[feature_cols]
y = pbp_2014_RP.rush
logreg.fit(X, y)
assorted_pred_class = logreg.predict(X)

print logreg.intercept_
zip(feature_cols, logreg.coef_[0])

In [ ]:
import statsmodels.formula.api as smf
lm = smf.ols(formula='rush ~ dwn + ytg + yfog', data=pbp_2014_RP).fit()

So far, this is all I've completed. I'd definitely like to get some feedback on the Logistic regression, and if the above code is logically sound. I plan to incorporate more models once we cover them in class. There is a ton of information that can be taken into account in these models - things like weather conditions, linemen statistics (including each individual offensive lineman present in each play), NFL combine data, run/pass directions, and a column indicating if a play was a "success" or not. The full definition of these terms is included in the pdf zip file. I'd like to dive in deeper and improve the accuracy, and also clean up the Data Visualization pieces and make them easier to look at. Thanks!